# Pattern Sanity Check on CORE Abstracts

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json

with open("downloads/sample_4k.json") as f:
    data = json.loads(f.read())
    #print(data['data'][0])

with open("ngrams_pos_only_lemmatized.txt") as f:
    ngrams = f.read().split("\n")
    
d = dict(data)

abst = [entry for entry in d["data"] if entry["abstract"]]

len(abst), ngrams[:3], abst[0]

(2956,
 ['with this', 'problem with', 'main drawback'],
 {'doi': '10.1029/2006GL027886',
  'coreId': '71102',
  'oai': 'oai:eprints.lancs.ac.uk:6706',
  'identifiers': ['oai:eprints.lancs.ac.uk:6706', '10.1029/2006GL027886'],
  'title': 'Transport of plasma sheet material to the inner magnetosphere',
  'authors': ['Denton, M. H.',
   'Thomsen, M. F.',
   'Borovsky, J. E.',
   'Lavraud, B.',
   'Henderson, M. G.',
   'Skoug, R. M.',
   'Funsten, H. O.',
   'Jahn, J.M.',
   'Pollock, C. J.',
   'Weygand, J. M.'],
  'enrichments': {'references': [], 'documentType': {'type': None}},
  'contributors': [],
  'datePublished': '2007',
  'abstract': 'The reaction of the plasma sheet in response to an increase in magnetospheric convection is examined using a combination of energetic neutral atom (ENA) imaging and in situ observations. Data from the IMAGE/MENA instrument are examined in conjunction with observations from the magnetospheric plasma analyzer (MPA) instrument onboard the Los Alamos 1

In [ ]:
trigrams = [gram for gram in ngrams if len(gram.split(" "))> 2]
bigrams = [gram for gram in ngrams if len(gram.split(" ")) == 2]

In [ ]:
from octoflow.core import lemmatize, find_sentence_in_abstract, split_into_sentences, sentence_has_phrase
import pandas as pd

In [ ]:
df = pd.DataFrame(abst)[["title", "doi", "abstract", "journals"]] #fulltext

In [ ]:
df["lemma_text"] = df["abstract"].map(lemmatizetize)

In [ ]:
df.to_csv("downloads/core_3k_lemma_abst_only.csv")

In [ ]:
hits = df[df["lemma_text"].str.contains("|".join(ngrams))]

In [ ]:
hits

,title,doi,abstract,journals,lemma_text
11,The single-payer option: a reconsideration,10.1215/03616878-2009-013,This article discusses some of the merits and ...,None,this article discuss some of the merit and dem...
38,A systematic review of the experiences and per...,10.1016/j.nedt.2009.10.017,Aim: A systematic literature review of the exp...,None,aim : a systematic literature review of the ex...
44,Talking with pictures: Exploring the possibili...,10.1080/0968776930010104,As multimedia computing becomes the order of t...,None,as multimedia computing become the order of th...
57,Cost Model for Evaluation of SDMB Service over...,10.1109/ISWCS.2005.1547789,The number of broadband users has beengrowing ...,None,the number of broadband user have beengrowe ra...
61,Renewable hydrogen utilisation for the product...,10.1016/j.enconman.2006.06.011,Electrolytic hydrogen production is an efficie...,"[{'title': None, 'identifiers': ['issn:0196-89...",electrolytic hydrogen production be an efficie...
...,...,...,...,...,...
2887,Error management in ATLAS TDAQ : an intelligen...,None,This thesis is concerned with the use of intel...,None,this thesis be concern with the use of intelli...
2914,International economic analysis of primary pre...,10.1053/euhj.1999.1193,Aims: The results of the West of Scotland Coro...,None,aim : the result of the West of Scotland Coron...
2927,Including older people with dementia in resear...,10.1080/1360786031000150685,NoThis paper examines key challenges and strat...,None,NoThis paper examine key challenge and strateg...
2930,Leadership in the European Convention,10.1080/13501760701656437,"Did the Chairman of the European Convention, V...",None,"do the Chairman of the European Convention , V..."


In [ ]:
from pydash import find_index

def find_sentence_in_abstract(paragraph, bias=0):
    """checks paragraph for key sentences. Returns first matching hit"""
    p_sents = split_into_sentences(paragraph)
    idx = find_index(p_sents, sentence_has_phrase)
    if idx < 0:
        return ""
    
    i = idx + bias #bias- for sentence before or after
    if i < 0:
        return ""
    return get(p_sents, i)

In [ ]:
def get_first_with_pattern(paragraph, patterns):
    for s in split_into_sentences(paragraph):
        if any(substring in s for substring in patterns):
            return s
    return ""
#df[df["lemma_text"].str.contains("|".join(ngrams))]

In [ ]:
mask1 = [get_first_with_pattern(abst, ngrams) != "" for abst in df["lemma_text"]]

In [ ]:
from re import search
mask2 = [search("|".join(ngrams), abst)  is not None for abst in df["lemma_text"] ]

In [ ]:
df[mask1]["title"].isin(df[mask2]["title"])

11      True
38      True
44      True
57      True
61      True
        ... 
2887    True
2914    True
2927    True
2930    True
2942    True
Name: title, Length: 244, dtype: bool

In [ ]:
lemmatize("Ph.D")

'Ph . D'

In [ ]:
hits["lemma_text"][11]

'this article discuss some of the merit and demerit of the single - payer model of health care financing , with particular reference to the English National Health Service ( NHS ) . specifically , it be argue that the main merit be that the model can directly provide universal health care coverage , thus eradicate or at least alleviate market failure and equity concern , and that it can achieve this with relatively low total health care expenditure in general and — as compare to the commercial multiple insurance model — low administrative cost in particular . a perceive demerit of the single - payer model be that it can lead to excessive health care rationing , particularly in term of waiting time , although it be argue here that long wait be probably cause by insufficient funding rather than by the single - payer model per se . moreover , rationing of one form or another occur in all health care system , and single - payer model may be the good option if the aim be to incorporate stru

In [ ]:
candidates = [entry for entry in abst if any(substring in entry["abstract"] for substring in ngrams)]